# Update Table Custom Property

This notebook provides an example for updated a table custom property. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update variable.

In [ ]:

import requests

API_KEY = ""
SECODA_API_URL = "https://api.secoda.co"

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))


def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

## Parse your content
Use a csv file that has the following columns: database, schema, table, column, property, value. If you'd like to just update table custom properties, leave the column `column` blank.

In [21]:
import pandas

dataframe = pandas.read_csv("./custom_properties.csv")
dataframe = dataframe.where(pandas.notnull(dataframe), None)

## Update table custom property
In this step, we find the table using the title, schema, and database and then update a custom property on the table.

In [ ]:
def find_column(column, table, schema, database, *args, **kwargs):
    res = session.get(build_url(f"/table/columns/?title={column}&parent__title={table}&parent__parent__title={schema}&parent__parent__parent__title={database}"))
    results = res.json().get("results")
    if len(results) >= 1:
        return results[0]
    return None


def set_column_properties(unique_id, properties):
    session.put(
        build_url(f"/table/columns/{unique_id}/"),
        json=dict(
            properties=properties,
        ),
    )

def find_table(table, schema, database, *args, **kwargs):
    res = session.get(build_url(f"/table/tables/?title={table}&parent__title={schema}&parent__parent__title={database}"))
    results = res.json().get("results")
    if len(results) >= 1:
        return results[0]
    return None


def set_table_properties(unique_id, properties):
    session.put(
        build_url(f"/table/tables/{unique_id}/"),
        json=dict(
            properties=properties,
        ),
    )

for index, entry in enumerate(dataframe.to_dict("records")):
    print(f"Processing {index}/{dataframe.shape[0]}")
    property = entry.get("property")
    value = entry.get("value")

    if entry.get("column", None) and entry.get("column", None) != "":
        column = find_column(**entry)
        if column:
            print(f"Found column: {column.get('title')}")
            properties = column.get("properties", {})
            if not "custom" in properties:
                properties["custom"] = {}

            properties["custom"][property] = value
            set_column_properties(column.get("id"), properties)
    else:
        table = find_table(**entry)
        if table:
            print(f"Found table: {table.get('title')}")
            properties = table.get("properties", {})
            if not "custom" in properties:
                properties["custom"] = []

            properties["custom"].append({
                'name': property,
                'value': value,
            })
            set_table_properties(table.get("id"), properties)
            
